In [48]:
import pandas as pd
from datetime import datetime
import numpy as np

In [49]:
path_ratings= "/home/suban/my_projct_dir/data/ml-100k/u.data"
path_movies= "/home/suban/my_projct_dir/data/ml-100k/u.item"
path_users= "/home/suban/my_projct_dir/data/ml-100k/u.user"

In [50]:
ratings_data= pd.read_csv(path_ratings, sep= "\t", header= None)
movies_data= pd.read_csv(path_movies, sep= "|", header= None, engine='python')
users_data= pd.read_csv(path_users, sep= "|", header= None)

In [51]:
movies_Header= ["Movie Id", "Movie Title", "Release Date", "Video Release Date",
                  "IMBd Url", "unknown", "Action", "Adventure", "Animation",
                   "Childen's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
                   "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-fi",
                   "Thriller", "War", "Western"]
ratings_Header= ["User Id", "Movie Id", "Rating", "Timestamp"]
users_Header= ["User Id", "Age", "Gender", "Occupation", "Zip Code"]

In [52]:
ratings_data.columns= ratings_Header
movies_data.columns= movies_Header
users_data.columns= users_Header

1-2: Print top 10 movies by ratings

In [54]:
movies= movies_data[["Movie Id", "Movie Title"]]
group= ratings_data[["Movie Id", "Rating"]].groupby("Movie Id")
mostRatings= group.count().nlargest(10,"Rating")
pd.merge(mostRatings, movies, on="Movie Id")

,Movie Id,Rating,Movie Title
0,50,583,Star Wars (1977)
1,258,509,Contact (1997)
2,100,508,Fargo (1996)
3,181,507,Return of the Jedi (1983)
4,294,485,Liar Liar (1997)
5,286,481,"English Patient, The (1996)"
6,288,478,Scream (1996)
7,1,452,Toy Story (1995)
8,300,431,Air Force One (1997)
9,121,429,Independence Day (ID4) (1996)


3: Print list of the number of ratings by genre

In [45]:
merged_data= pd.merge(ratings_data, movies_data, left_on="Movie Id", right_on="Movie Id").reindex()
merged_data[merged_data.columns[8:]].sum()

unknown           10
Action         25589
Adventure      13753
Animation       3605
Childen's       7182
Comedy         29832
Crime           8055
Documentary      758
Drama          39895
Fantasy         1352
Film-Noir       1733
Horror          5317
Musical         4954
Mystery         5245
Romance        19461
Sci-fi         12730
Thriller       21872
War             9398
Western         1854
dtype: int64

4: Print the oldest movie with a "5" rating

In [29]:
print(ratings_data)

       User Id  Movie Id  Rating  Timestamp
0          196       242       3  881250949
1          186       302       3  891717742
2           22       377       1  878887116
3          244        51       2  880606923
4          166       346       1  886397596
...        ...       ...     ...        ...
99995      880       476       3  880175444
99996      716       204       5  879795543
99997      276      1090       1  874795795
99998       13       225       2  882399156
99999       12       203       3  879959583

[100000 rows x 4 columns]


In [30]:
ratings= ratings_data[["Movie Id", "Rating"]]
merged_data= pd.merge(ratings, movies_data, left_on="Movie Id", right_on="Movie Id").reindex()
merged_data= merged_data[["Movie Id","Movie Title", "Rating", "Release Date"]]
merged_data= merged_data.loc[merged_data["Rating"]==5]
merged_data["Release Date"]= pd.to_datetime(merged_data["Release Date"])
merged_data= merged_data.sort_values(by="Release Date")
print(merged_data["Movie Title"].iloc[1])

Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922)


5: Printing the genres of the top 10 most rated movies

In [31]:
merged_data=pd.merge(mostRatings, movies_data,left_on="Movie Id", right_on="Movie Id")
genres= merged_data.loc[:,"unknown":"Western"]
merged_data["Genres"]= 1

movies= ratings_data["Movie Id"].value_counts().index[:10]
for i in movies:
    title= movies_data[movies_data["Movie Id"]== i]["Movie Title"].values[0]
    genres= movies_data[movies_data["Movie Id"]== i].iloc[:,5:]
    list= genres.columns[(genres ==1).values[0]]
    print("{:30} genres: {}".format(title, ", ".join(list)))

Star Wars (1977)               genres: Action, Adventure, Romance, Sci-fi, War
Contact (1997)                 genres: Drama, Sci-fi
Fargo (1996)                   genres: Crime, Drama, Thriller
Return of the Jedi (1983)      genres: Action, Adventure, Romance, Sci-fi, War
Liar Liar (1997)               genres: Comedy
English Patient, The (1996)    genres: Drama, Romance, War
Scream (1996)                  genres: Horror, Thriller
Toy Story (1995)               genres: Animation, Childen's, Comedy
Air Force One (1997)           genres: Action, Thriller
Independence Day (ID4) (1996)  genres: Action, Sci-fi, War


6: Print the title of the movie that was rated the most by students

In [32]:
students= users_data.loc[users_data["Occupation"]== "student"]["User Id"]
ratings= ratings_data[ratings_data["User Id"].isin(students)]
movie= int(ratings_data["Movie Id"].value_counts().head(1))
movies_data.loc[movies_data["Movie Id"]== movie]["Movie Title"]

582    Romeo Is Bleeding (1993)
Name: Movie Title, dtype: object


7: Print the list of movies that received the highest number of "5" rating

In [33]:
ratings5Id= ratings_data.loc[ratings_data["Rating"]==5]["Movie Id"].value_counts().index
for i in ratings5Id[:10]:
    print(movies_data[movies_data["Movie Id"]== i]["Movie Title"].values[0])

Star Wars (1977)
Fargo (1996)
Godfather, The (1972)
Raiders of the Lost Ark (1981)
Pulp Fiction (1994)
Schindler's List (1993)
Silence of the Lambs, The (1991)
Titanic (1997)
Empire Strikes Back, The (1980)
Return of the Jedi (1983)


8: Print list of zip codes corresponding to the highest number of users that rated movies

In [35]:
user= set(ratings_data["User Id"])
rated_user= users_data[users_data["User Id"].isin(user)]
group= rated_user.groupby("Zip Code")["User Id"].count()
top10= group.nlargest(10)
print(top10)

Zip Code
55414    9
55105    6
10003    5
20009    5
55337    5
27514    4
55408    4
55454    4
02215    3
10021    3
Name: User Id, dtype: int64



9: Find the most rated movie by users in the age group 20-25

In [36]:
print(users_data)

     User Id  Age Gender     Occupation Zip Code
0          1   24      M     technician    85711
1          2   53      F          other    94043
2          3   23      M         writer    32067
3          4   24      M     technician    43537
4          5   33      F          other    15213
..       ...  ...    ...            ...      ...
938      939   26      F        student    33319
939      940   32      M  administrator    02215
940      941   20      M        student    97229
941      942   48      F      librarian    78209
942      943   22      M        student    77841

[943 rows x 5 columns]


In [37]:
users= users_data[users_data["Age"].between(20,25)]
users_ids= users["User Id"]
ratings= ratings_data[ratings_data["User Id"].isin(users_ids)]
movie_id= ratings["Movie Id"].value_counts().index[0]
movie= movies_data[movies_data["Movie Id"]==movie_id]["Movie Title"]
print(movie)

287    Scream (1996)
Name: Movie Title, dtype: object


In [38]:
ratings_data["Date"]= ratings_data["Timestamp"].apply(lambda x: datetime.fromtimestamp(x))
ratings= ratings_data[ratings_data["Date"] > datetime(1960,12,31)]
movie_Ids= ratings["Movie Id"]
movies= movies_data[movies_data["Movie Id"].isin(movie_Ids)]
print(movies["Movie Title"][:10])

0                                     Toy Story (1995)
1                                     GoldenEye (1995)
2                                    Four Rooms (1995)
3                                    Get Shorty (1995)
4                                       Copycat (1995)
5    Shanghai Triad (Yao a yao yao dao waipo qiao) ...
6                                Twelve Monkeys (1995)
7                                          Babe (1995)
8                              Dead Man Walking (1995)
9                                   Richard III (1995)
Name: Movie Title, dtype: object
